[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/geemap/blob/master/docs/workshops/Alaska_2024_Part3.ipynb)

**Geospatial Cloud Computing with the GEE Python API - Part 3**

-   Notebook: <https://geemap.org/workshops/Alaska_2024_Part3>
-   Earth Engine: <https://earthengine.google.com>
-   Geemap: <https://geemap.org>

## Introduction

This notebook contains the materials for the third part of the workshop **Geospatial Cloud Computing with the GEE Python API** at the University of Alaska Fairbanks.

This workshop provides an introduction to cloud-based geospatial analysis using the Earth Engine Python API. Attendees will learn the basics of Earth Engine data types and how to visualize, analyze, and export Earth Engine data in a Jupyter environment with geemap. In addition, attendees will learn how to develop and deploy interactive Earth Engine web apps with Python. Through practical examples and hands-on exercises, attendees will enhance their learning experience. During each hands-on session, attendees will walk through Jupyter Notebook examples on Google Colab with the instructors. At the end of each session, they will complete a hands-on exercise to apply the knowledge they have learned.

### Agenda

The workshop is divided into three parts. The third part will cover the following topics:

-   Image Classification (focused on land cover in Alaska)
-   Accuracy assessment
-   Create and export maps
-   Building interactive web apps

### Prerequisites

-   To use geemap and the Earth Engine Python API, you must [register](https://code.earthengine.google.com/register) for an Earth Engine account and follow the instructions [here](https://docs.google.com/document/d/1ZGSmrNm6_baqd8CHt33kIBWOlvkh-HLr46bODgJN1h0/edit?usp=sharing) to create a Cloud Project. Earth Engine is free for [noncommercial and research use](https://earthengine.google.com/noncommercial). To test whether you can use authenticate the Earth Engine Python API, please run [this notebook](https://colab.research.google.com/github/giswqs/geemap/blob/master/examples/notebooks/geemap_colab.ipynb) on Google Colab.

### Import libraries

In [ ]:
import ee
import geemap

In [ ]:
geemap.ee_initialize()

## Image Classification

### Unsupervised classification

In [ ]:
m = geemap.Map()

point = ee.Geometry.Point([-88.0664, 41.9411])

image = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2022-01-01", "2022-12-31")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

region = image.geometry()
image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {"min": 0, "max": 0.3, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

m.centerObject(region, 8)
m.addLayer(image, vis_params, "Landsat-9")
m

In [ ]:
geemap.get_info(image)

In [ ]:
image.get("DATE_ACQUIRED").getInfo()

In [ ]:
image.get("CLOUD_COVER").getInfo()

In [ ]:
training = image.sample(
    **{
        # "region": region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

m.addLayer(training, {}, "Training samples")
m

In [ ]:
geemap.ee_to_df(training.limit(5))

In [ ]:
n_clusters = 5
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

In [ ]:
result = image.cluster(clusterer)
m.addLayer(result.randomVisualizer(), {}, "clusters")
m

In [ ]:
legend_dict = {
    "Open Water": "#466b9f",
    "Developed, High Intensity": "#ab0000",
    "Developed, Low Intensity": "#d99282",
    "Forest": "#1c5f2c",
    "Cropland": "#ab6c28",
}

palette = list(legend_dict.values())

m.addLayer(result, {"min": 0, "max": 4, "palette": palette}, "Labelled clusters")
m.add_legend(title="Land Cover Type", legend_dict=legend_dict, position="bottomright")
m

In [ ]:
geemap.download_ee_image(image, filename="unsupervised.tif", region=region, scale=90)

### Supervised classification

In [ ]:
m = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

image = (
    ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
    .filterBounds(point)
    .filterDate("2019-01-01", "2020-01-01")
    .sort("CLOUD_COVER")
    .first()
    .select("SR_B[1-7]")
)

image = image.multiply(0.0000275).add(-0.2).set(image.toDictionary())
vis_params = {"min": 0, "max": 0.3, "bands": ["SR_B5", "SR_B4", "SR_B3"]}

m.centerObject(point, 8)
m.addLayer(image, vis_params, "Landsat-8")
m

In [ ]:
geemap.get_info(image)

In [ ]:
image.get("DATE_ACQUIRED").getInfo()

In [ ]:
image.get("CLOUD_COVER").getInfo()

In [ ]:
nlcd = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019")
landcover = nlcd.select("landcover").clip(image.geometry())
m.addLayer(landcover, {}, "NLCD Landcover")
m

In [ ]:
points = landcover.sample(
    **{
        "region": image.geometry(),
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,
    }
)

m.addLayer(points, {}, "training", False)

In [ ]:
print(points.size().getInfo())

In [ ]:
bands = ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7"]
label = "landcover"
features = image.select(bands).sampleRegions(
    **{"collection": points, "properties": [label], "scale": 30}
)

In [ ]:
geemap.ee_to_df(features.limit(5))

In [ ]:
params = {
    "features": features,
    "classProperty": label,
    "inputProperties": bands,
}
classifier = ee.Classifier.smileCart(maxNodes=None).train(**params)

In [ ]:
classified = image.select(bands).classify(classifier).rename("landcover")
m.addLayer(classified.randomVisualizer(), {}, "Classified")
m

In [ ]:
geemap.get_info(nlcd)

In [ ]:
class_values = nlcd.get("landcover_class_values")
class_palette = nlcd.get("landcover_class_palette")
classified = classified.set(
    {"landcover_class_values": class_values, "landcover_class_palette": class_palette}
)

In [ ]:
m.addLayer(classified, {}, "Land cover")
m.add_legend(title="Land cover type", builtin_legend="NLCD")
m

In [ ]:
geemap.download_ee_image(
    landcover, filename="supervised.tif", region=image.geometry(), scale=30
)

## Accuracy assessment

In [ ]:
m = geemap.Map()
point = ee.Geometry.Point([-122.4439, 37.7538])

img = (
    ee.ImageCollection("COPERNICUS/S2_SR")
    .filterBounds(point)
    .filterDate("2020-01-01", "2021-01-01")
    .sort("CLOUDY_PIXEL_PERCENTAGE")
    .first()
    .select("B.*")
)

vis_params = {"min": 100, "max": 3500, "bands": ["B11", "B8", "B3"]}

m.centerObject(point, 9)
m.addLayer(img, vis_params, "Sentinel-2")
m

In [ ]:
lc = ee.Image("ESA/WorldCover/v100/2020")
classValues = [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100]
remapValues = ee.List.sequence(0, 10)
label = "lc"
lc = lc.remap(classValues, remapValues).rename(label).toByte()

In [ ]:
sample = img.addBands(lc).stratifiedSample(
    **{
        "numPoints": 100,
        "classBand": label,
        "region": img.geometry(),
        "scale": 10,
        "geometries": True,
    }
)

In [ ]:
sample = sample.randomColumn()
trainingSample = sample.filter("random <= 0.8")
validationSample = sample.filter("random > 0.8")

In [ ]:
trainedClassifier = ee.Classifier.smileRandomForest(numberOfTrees=10).train(
    **{
        "features": trainingSample,
        "classProperty": label,
        "inputProperties": img.bandNames(),
    }
)

In [ ]:
print("Results of trained classifier", trainedClassifier.explain().getInfo())

In [ ]:
trainAccuracy = trainedClassifier.confusionMatrix()
trainAccuracy.getInfo()

In [ ]:
trainAccuracy.accuracy().getInfo()

In [ ]:
trainAccuracy.kappa().getInfo()

In [ ]:
validationSample = validationSample.classify(trainedClassifier)
validationAccuracy = validationSample.errorMatrix(label, "classification")
validationAccuracy.getInfo()

In [ ]:
validationAccuracy.accuracy().getInfo()

In [ ]:
validationAccuracy.producersAccuracy().getInfo()

In [ ]:
validationAccuracy.consumersAccuracy().getInfo()

In [ ]:
import csv

with open("training.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainAccuracy.getInfo())

with open("validation.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(validationAccuracy.getInfo())

In [ ]:
imgClassified = img.classify(trainedClassifier)

In [ ]:
classVis = {
    "min": 0,
    "max": 10,
    "palette": [
        "006400",
        "ffbb22",
        "ffff4c",
        "f096ff",
        "fa0000",
        "b4b4b4",
        "f0f0f0",
        "0064c8",
        "0096a0",
        "00cf75",
        "fae6a0",
    ],
}
m.addLayer(lc, classVis, "ESA Land Cover", False)
m.addLayer(imgClassified, classVis, "Classified")
m.addLayer(trainingSample, {"color": "black"}, "Training sample")
m.addLayer(validationSample, {"color": "white"}, "Validation sample")
m.add_legend(title="Land Cover Type", builtin_legend="ESA_WorldCover")
m.centerObject(img)
m

## Create and export maps

In [ ]:
m = geemap.Map(center=(41.0462, -109.7424), zoom=6)

dem = ee.Image("USGS/SRTMGL1_003")
landsat7 = ee.Image("LANDSAT/LE7_TOA_5YEAR/1999_2003").select(
    ["B1", "B2", "B3", "B4", "B5", "B7"]
)

vis_params = {
    "min": 0,
    "max": 4000,
    "palette": ["006633", "E5FFCC", "662A00", "D8D8D8", "F5F5F5"],
}

m.add_layer(
    landsat7,
    {"bands": ["B4", "B3", "B2"], "min": 20, "max": 200, "gamma": 2},
    "landsat",
)
m.add_layer(dem, vis_params, "dem", True, 1)
m

In [ ]:
m.layer_to_image("dem", output="dem.tif", crs="EPSG:3857", region=None, scale=None)
m.layer_to_image("dem", output="dem.jpg", scale=500)
geemap.show_image("dem.jpg")

In [ ]:
m.layer_to_image("landsat", output="landsat.tif")
geemap.geotiff_to_image("landsat.tif", output="landsat.jpg")
geemap.show_image("landsat.jpg")

In [ ]:
from geemap import cartoee
import matplotlib.pyplot as plt

### Plotting single-band images

In [ ]:
srtm = ee.Image("CGIAR/SRTM90_V4")

# define bounding box [east, south, west, north] to request data
region = [180, -60, -180, 85]
vis = {"min": 0, "max": 3000}

In [ ]:
fig = plt.figure(figsize=(15, 9))

# use cartoee to get a map
ax = cartoee.get_map(srtm, region=region, vis_params=vis)

# add a color bar to the map using the visualization params we passed to the map
cartoee.add_colorbar(
    ax, vis, loc="bottom", label="Elevation (m)", orientation="horizontal"
)

# add grid lines to the map at a specified interval
cartoee.add_gridlines(ax, interval=[60, 30], linestyle=":")

# add coastlines using the cartopy api
ax.coastlines(color="red")

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 7))

cmap = "terrain"

ax = cartoee.get_map(srtm, region=region, vis_params=vis, cmap=cmap)
cartoee.add_colorbar(
    ax, vis, cmap=cmap, loc="right", label="Elevation (m)", orientation="vertical"
)

cartoee.add_gridlines(ax, interval=[60, 30], linestyle="--")
ax.coastlines(color="red")
ax.set_title(label="Global Elevation Map", fontsize=15)

plt.show()

In [ ]:
cartoee.savefig(fig, fname="srtm.jpg", dpi=300, bbox_inches="tight")

### Plotting multi-band images

In [ ]:
image = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_044034_20140318")
vis = {"bands": ["B5", "B4", "B3"], "min": 0, "max": 5000, "gamma": 1.3}

In [ ]:
fig = plt.figure(figsize=(15, 10))

ax = cartoee.get_map(image, vis_params=vis)
cartoee.pad_view(ax)
cartoee.add_gridlines(ax, interval=0.5, xtick_rotation=0, linestyle=":")
ax.coastlines(color="yellow")

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))

region = [-121.8025, 37.3458, -122.6265, 37.9178]
ax = cartoee.get_map(image, vis_params=vis, region=region)
cartoee.add_gridlines(ax, interval=0.15, xtick_rotation=0, linestyle=":")
ax.coastlines(color="yellow")

plt.show()

### Using custom projections

#### The PlateCarree projection

In [ ]:
ocean = (
    ee.ImageCollection("NASA/OCEANDATA/MODIS-Terra/L3SMI")
    .filter(ee.Filter.date("2018-01-01", "2018-03-01"))
    .median()
    .select(["sst"], ["SST"])
)

In [ ]:
visualization = {"bands": "SST", "min": -2, "max": 30}
bbox = [180, -88, -180, 88]

In [ ]:
fig = plt.figure(figsize=(15, 10))

ax = cartoee.get_map(ocean, cmap="plasma", vis_params=visualization, region=bbox)
cb = cartoee.add_colorbar(ax, vis_params=visualization, loc="right", cmap="plasma")

ax.set_title(label="Sea Surface Temperature", fontsize=15)

ax.coastlines()
plt.show()

In [ ]:
cartoee.savefig(fig, "SST.jpg", dpi=300)

#### Custom projections

In [ ]:
import cartopy.crs as ccrs

In [ ]:
fig = plt.figure(figsize=(15, 10))

projection = ccrs.Mollweide(central_longitude=-180)
ax = cartoee.get_map(
    ocean, vis_params=visualization, region=bbox, cmap="plasma", proj=projection
)
cb = cartoee.add_colorbar(
    ax, vis_params=visualization, loc="bottom", cmap="plasma", orientation="horizontal"
)
ax.set_title("Mollweide projection")
ax.coastlines()

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))

projection = ccrs.Robinson(central_longitude=-180)
ax = cartoee.get_map(
    ocean, vis_params=visualization, region=bbox, cmap="plasma", proj=projection
)
cb = cartoee.add_colorbar(
    ax, vis_params=visualization, loc="bottom", cmap="plasma", orientation="horizontal"
)
ax.set_title("Robinson projection")
ax.coastlines()

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))

projection = ccrs.InterruptedGoodeHomolosine(central_longitude=-180)
ax = cartoee.get_map(
    ocean, vis_params=visualization, region=bbox, cmap="plasma", proj=projection
)
cb = cartoee.add_colorbar(
    ax, vis_params=visualization, loc="bottom", cmap="plasma", orientation="horizontal"
)
ax.set_title("Goode homolosine projection")
ax.coastlines()

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))

projection = ccrs.EqualEarth(central_longitude=-180)
ax = cartoee.get_map(
    ocean, vis_params=visualization, region=bbox, cmap="plasma", proj=projection
)
cb = cartoee.add_colorbar(
    ax, vis_params=visualization, loc="right", cmap="plasma", orientation="vertical"
)
ax.set_title("Equal Earth projection")
ax.coastlines()

plt.show()

In [ ]:
fig = plt.figure(figsize=(11, 10))

projection = ccrs.Orthographic(-130, -10)
ax = cartoee.get_map(
    ocean, vis_params=visualization, region=bbox, cmap="plasma", proj=projection
)
cb = cartoee.add_colorbar(
    ax, vis_params=visualization, loc="right", cmap="plasma", orientation="vertical"
)
ax.set_title("Orographic projection")
ax.coastlines()

plt.show()

## Building interactive web apps

```bash
# %pip install solara
```

In [ ]:
import ee
import geemap
import solara


class Map(geemap.Map):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.add_ee_data()

    def add_ee_data(self):
        years = ["2001", "2004", "2006", "2008", "2011", "2013", "2016", "2019"]

        def getNLCD(year):
            dataset = ee.ImageCollection("USGS/NLCD_RELEASES/2019_REL/NLCD")
            nlcd = dataset.filter(ee.Filter.eq("system:index", year)).first()
            landcover = nlcd.select("landcover")
            return landcover

        collection = ee.ImageCollection(ee.List(years).map(lambda year: getNLCD(year)))
        labels = [f"NLCD {year}" for year in years]
        self.ts_inspector(
            left_ts=collection,
            right_ts=collection,
            left_names=labels,
            right_names=labels,
        )
        self.add_legend(
            title="NLCD Land Cover Type",
            builtin_legend="NLCD",
            height="460px",
            add_header=False,
        )


@solara.component
def Page():
    with solara.Column(style={"min-width": "500px"}):
        m.element(
            center=[40, -100],
            zoom=4,
            height="800px",
        )

```bash
conda activate gee
solara run ./pages
```

In [ ]:
import geemap

geemap.get_ee_token()